## Exploring different Document Processing Models
 
 **Goal**: An Agentic approach to explore different Document Processing Models for OCR, Layout Analysis, and Table Extraction.
 
 **Output**: A simple langgraph agent that explores different ocr techniques/models.


In [1]:
# Setting up a react agent with LangChain


from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import AIMessageChunk, HumanMessage
from langchain_core.runnables import RunnableConfig

# from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import InMemorySaver
from langsmith import traceable
from prompts import DOCUMENT_AGENT_SYSTEM_PROMPT, USER_MESSAGE_TEMPLATE

load_dotenv()
# conn = connect("langgraph_checkpoints.db")
# checkpointer = SqliteSaver(conn=conn)
checkpointer = InMemorySaver()
llm = init_chat_model(
    "anthropic:claude-sonnet-4-5-20250929", temperature=1, thinking={"type": "enabled", "budget_tokens": 10000}
)

document_agent = create_agent(
    model=llm,
    system_prompt=DOCUMENT_AGENT_SYSTEM_PROMPT,
    checkpointer=checkpointer,
)
config = RunnableConfig(configurable={"thread_id": "test_thread"})
inputs = {
    "messages": [
        HumanMessage(
            USER_MESSAGE_TEMPLATE.format(message="What OCR tools are available?", document_path="No Document Uploaded")
        )
    ]
}


@traceable(name="DocumentAgentInteraction")
def agent_execution():
    """Execute the document agent and streams the response."""
    ai_message = AIMessageChunk(content=[])

    for _, chunk in document_agent.stream(inputs, stream_mode=["messages"], config=config):
        ai_message += chunk[0]

    ai_message.pretty_print()
    return ai_message


ai_message = agent_execution()

/home/talha/Desktop/Work repositories/Document_Processing/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
/home/talha/Desktop/Work repositories/Document_Processing/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


============================ Aimessagechunk Message ============================

[{'thinking': 'The user is asking what OCR tools are available. Looking at the context provided, I can see that there\'s a placeholder for "{available_ocr_tools}" in the system instructions, but this hasn\'t been filled in with actual tools.\n\nThe user also mentions they want to process a document using all available OCR techniques, but the system message clearly states "No Document Uploaded".\n\nI should:\n1. Acknowledge that no document has been uploaded yet\n2. Explain that I would need to know what OCR tools are configured in the system (the {available_ocr_tools} placeholder suggests this should be provided by the system)\n3. Ask them to upload a document first before we can process it\n\nSince I don\'t have information about the actual OCR tools available (the placeholder wasn\'t replaced), I cannot list specific tools. I should be honest about this limitation.', 'type': 'thinking', 'index': 0, 'sig

In [3]:
from datetime import datetime
from uuid import UUID

uuid = UUID(ai_message.id.split("--")[1])
ts_ms = uuid.int >> 80
ts_s = ts_ms / 1000
dt = datetime.fromtimestamp(ts_s)
print("Message created at:", dt)

Message created at: 2026-01-19 21:26:13.114000


In [ ]:
from langchain.messages import HumanMessage
from langchain_community.chat_models import ChatSnowflakeCortex

config = {
    # add the required Snowflake connection parameters here
}
chat = ChatSnowflakeCortex(
    snowflake_username=config["user"],
    snowflake_password=config["password"],
    snowflake_account=config["account"],
    snowflake_warehouse=config["warehouse"],
    snowflake_database=config["database"],
    snowflake_schema=config["schema"],
    snowflake_role=config["role"],
)

/home/talha/Desktop/Work repositories/Document_Processing/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
/home/talha/Desktop/Work repositories/Document_Processing/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chat.invoke("hi")

AIMessage(content=' Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20}, id='lc_run--019c2e20-dcfb-71f0-aafb-1f29c7285e4b-0', tool_calls=[], invalid_tool_calls=[])